In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from keras.layers import Dense, Input
from keras.models import Sequential
from keras import optimizers

from sympy import Symbol

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [2]:
X, y = make_regression(n_samples=1000, n_features=4, n_informative=3, noise=0.5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print(lr.intercept_.round(2), *lr.coef_.round(2))
y_pred = lr.predict(X_test_scaled)
mean_squared_error(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred)

-297.7 0.01 341.84 204.73 39.37


(0.25597233101489053, 0.04145703894995921)

In [5]:
mlp = Sequential([
    Input((4,)),
    Dense(1, activation='linear')
])
mlp.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (20.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
mlp.compile(loss='mse', optimizer=optimizers.SGD(0.1), metrics=['mape'])
history = mlp.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_data=(X_test_scaled, y_test))

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3875.5554 - mape: 123.2759 - val_loss: 3009.7087 - val_mape: 93.8257
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2770.3145 - mape: 117.5653 - val_loss: 2234.2336 - val_mape: 98.3783
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1909.6105 - mape: 83.6888 - val_loss: 1631.4307 - val_mape: 96.9195
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1393.7181 - mape: 90.6272 - val_loss: 1195.4780 - val_mape: 94.2515
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1112.7397 - mape: 96.5329 - val_loss: 1084.0880 - val_mape: 127.9125
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 841.0449 - mape: 129.6881 - val_loss: 705.5593 - val_mape: 90.5838
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 679.0338 - mape: 65.3228 - val_loss: 577.0162 - val_mape: 91.1116
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 498.3183 - mape: 164.5477 - val_loss: 443.6562 - val_mape: 8

In [7]:
loss_data = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(loss_data, title='Loss (MSE)').show()

mape_data = pd.DataFrame({
    'train': history.history['mape'],
    'test': history.history['val_mape']
})

px.line(mape_data, title='MAPE').show()

In [10]:
for layer in mlp.layers:
    print(layer.get_weights())

[array([[ -0.95939493],
       [340.71048   ],
       [204.04048   ],
       [ 37.40663   ]], dtype=float32), array([-294.9747], dtype=float32)]
